# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,22.07,48,7,0.89,AU,2025-01-30
1,1,guerrero negro,27.9769,-114.0611,13.99,69,0,7.15,MX,2025-01-30
2,2,okha,53.5739,142.9478,-13.02,95,100,6.95,RU,2025-01-30
3,3,grytviken,-54.2811,-36.5092,4.66,65,83,3.89,GS,2025-01-30
4,4,port-aux-francais,-49.3500,70.2167,6.93,90,100,16.39,TF,2025-01-30


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
)

# Display the map
hvplot.save(humidity_map, "output_data/humidity_map.html")
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_cities_df = city_data_df[(city_data_df['Lat'] >= -30) & (city_data_df['Lat'] <= 30) & (city_data_df['Cloudiness'] <=20)]

# Drop any rows with null values
ideal_city_weather_df = narrow_cities_df.dropna()

# Display sample data
ideal_city_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,guerrero negro,27.9769,-114.0611,13.99,69,0,7.15,MX,2025-01-30
14,14,pisco,-13.7000,-76.2167,22.03,83,0,9.26,PE,2025-01-30
32,32,tolanaro,-25.0319,46.9987,25.16,88,1,3.63,MG,2025-01-30
36,36,saipan,15.1355,145.7010,31.39,73,0,9.26,MP,2025-01-30
44,44,afaahiti,-17.7500,-149.2833,26.64,77,19,6.95,PF,2025-01-30
...,...,...,...,...,...,...,...,...,...,...
549,549,bardai,11.6500,7.2333,17.53,23,2,4.75,NG,2025-01-30
550,550,atar,20.5169,-13.0499,17.25,23,0,4.61,MR,2025-01-30
559,559,siwa oasis,29.2041,25.5195,13.64,49,0,3.41,EG,2025-01-30
561,561,ouro sogui,15.6000,-13.3167,23.53,12,0,5.33,SN,2025-01-30


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_weather_df[['City', 'Country','Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

C:\Users\mrbow\AppData\Local\Temp\ipykernel_22056\538563850.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Country,Lat,Lng,Humidity,Hotel Name
1,guerrero negro,MX,27.9769,-114.0611,69,
14,pisco,PE,-13.7000,-76.2167,83,
32,tolanaro,MG,-25.0319,46.9987,88,
36,saipan,MP,15.1355,145.7010,73,
44,afaahiti,PF,-17.7500,-149.2833,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
params = {
    "apiKey": geoapify_key,
    "limit":limit, 
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
guerrero negro - nearest hotel: Plaza sal paraiso
pisco - nearest hotel: La Portada
tolanaro - nearest hotel: Hôtel Mahavokey
saipan - nearest hotel: Chalan Kanoa Beach Hotel
afaahiti - nearest hotel: Omati Lodge
ormara - nearest hotel: No hotel found
nova sintra - nearest hotel: Residência Ka Dencho
avarua - nearest hotel: Paradise Inn
kutum - nearest hotel: No hotel found
callao - nearest hotel: Casa Ronald
kismayo - nearest hotel: Kismayo Hotel
adi keyh - nearest hotel: ዓዲ ቐይሕ ሆቴል
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
georgetown - nearest hotel: Page 63 hostel
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
saint-pierre - nearest hotel: Tropic Hotel
bubaque - nearest hotel: Chez Julio
ubari - nearest hotel: فندق اوباري
akkarampalle - nearest hotel: Hotel Orion
la asuncion - nearest hotel: Hotel La Ciudad
gopalpur - nearest hotel: Swosti
cam ranh - nearest hotel: Khách sạn Thái Bình
kuraymah - nearest hotel: Hotel al Nasser
s

,City,Country,Lat,Lng,Humidity,Hotel Name
1,guerrero negro,MX,27.9769,-114.0611,69,Plaza sal paraiso
14,pisco,PE,-13.7000,-76.2167,83,La Portada
32,tolanaro,MG,-25.0319,46.9987,88,Hôtel Mahavokey
36,saipan,MP,15.1355,145.7010,73,Chalan Kanoa Beach Hotel
44,afaahiti,PF,-17.7500,-149.2833,77,Omati Lodge
...,...,...,...,...,...,...
549,bardai,NG,11.6500,7.2333,23,No hotel found
550,atar,MR,20.5169,-13.0499,23,Residence Egueil
559,siwa oasis,EG,29.2041,25.5195,49,Flower of the city hotel
561,ouro sogui,SN,15.6000,-13.3167,12,Hôtel Wilma


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Lat",
    scale = 1,
    color = "City", 
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hvplot.save(hotel_map, "output_data/hotel_map.html")
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)